In [1]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.applications.inception_v3 import InceptionV3
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras import optimizers, losses, activations, models
import itertools

# 資料路徑
DATASET_PATH  = 'data'

# 影像大小
IMAGE_SIZE = (272, 272)

# 影像類別數
NUM_CLASSES = 2

# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 10

# 凍結網路層數
FREEZE_LAYERS = 0

# Epoch 數
NUM_EPOCHS = 35



   

# 透過 data augmentation 產生訓練與驗證用的影像資料
train_datagen = ImageDataGenerator(rotation_range=30,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.2,
                                   zoom_range=0.1,
                                   channel_shift_range=15,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/train',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

valid_datagen = ImageDataGenerator()
valid_batches = valid_datagen.flow_from_directory(DATASET_PATH + '/valid',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE)

# 輸出各類別的索引值
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))



# 以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers
net = InceptionV3(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)

# 設定凍結與要進行訓練的網路層
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
net_final.compile(optimizer=Adam(lr=0.0001),
                  loss='categorical_crossentropy', metrics=['accuracy'])

# 輸出整個網路結構
print(net_final.summary())

file_path="08180738weights.best.hdf5"

checkpoint = ModelCheckpoint(file_path, monitor='acc', verbose=1, save_best_only=True, mode='max')

early = EarlyStopping(monitor="acc", mode="max", patience=15)

reduce_lr = ReduceLROnPlateau(factor=0.1, 
                              min_lr=1e-12, 
                              monitor='val_loss', 
                              patience=3, 
                              verbose=1)

callbacks_list = [checkpoint, early,reduce_lr] #early

# 訓練模型
net_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS,
                              shuffle=True, 
                              verbose=True,
                              callbacks=callbacks_list)




Found 3400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Class #0 = cats
Class #1 = dogs


W0818 12:15:09.348933  1900 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 272, 272, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 135, 135, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 135, 135, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 135, 135, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

Epoch 1/35
339/340 [============================>.] - ETA: 0s - loss: 0.2720 - acc: 0.9130
Epoch 00001: acc improved from -inf to 0.91265, saving model to 08180738weights.best.hdf5
340/340 [==============================] - 120s 353ms/step - loss: 0.2747 - acc: 0.9126 - val_loss: 0.1228 - val_acc: 0.9750
Epoch 2/35
339/340 [============================>.] - ETA: 0s - loss: 0.1203 - acc: 0.9540
Epoch 00002: acc improved from 0.91265 to 0.95412, saving model to 08180738weights.best.hdf5
340/340 [==============================] - 68s 201ms/step - loss: 0.1199 - acc: 0.9541 - val_loss: 0.0626 - val_acc: 0.9750
Epoch 3/35
339/340 [============================>.] - ETA: 0s - loss: 0.1013 - acc: 0.9681
Epoch 00003: acc improved from 0.95412 to 0.96765, saving model to 08180738weights.best.hdf5
340/340 [==============================] - 69s 202ms/step - loss: 0.1017 - acc: 0.9676 - val_loss: 0.1159 - val_acc: 0.9683
Epoch 4/35
339/340 [============================>.] - ETA: 0s - loss: 0.1256 -

Epoch 28/35
339/340 [============================>.] - ETA: 0s - loss: 0.0159 - acc: 0.9950
Epoch 00028: acc did not improve from 0.99794
340/340 [==============================] - 67s 197ms/step - loss: 0.0158 - acc: 0.9950 - val_loss: 0.0538 - val_acc: 0.9850
Epoch 29/35
339/340 [============================>.] - ETA: 0s - loss: 0.0106 - acc: 0.9965- ETA: 
Epoch 00029: acc did not improve from 0.99794

Epoch 00029: ReduceLROnPlateau reducing learning rate to 1e-12.
340/340 [==============================] - 67s 198ms/step - loss: 0.0108 - acc: 0.9962 - val_loss: 0.0520 - val_acc: 0.9833
Epoch 30/35
339/340 [============================>.] - ETA: 0s - loss: 0.0070 - acc: 0.9988
Epoch 00030: acc improved from 0.99794 to 0.99882, saving model to 08180738weights.best.hdf5
340/340 [==============================] - 68s 200ms/step - loss: 0.0070 - acc: 0.9988 - val_loss: 0.0508 - val_acc: 0.9833
Epoch 31/35
339/340 [============================>.] - ETA: 0s - loss: 0.0212 - acc: 0.9950
Epo

In [2]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.preprocessing import image
import sys
import numpy as np
import pandas as pd

# 從參數讀取圖檔路徑

import os

path = 'data\\test'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.jpg' in file:
            files.append(os.path.join(r, file))
print (files)

# 載入訓練好的模型
net = load_model(file_path)

cls_list = ['cats', 'dogs']
cols=['ID', 'Predicted']
df=pd.DataFrame(columns=cols)
# 辨識每一張圖
for f in files:
    img = image.load_img(f, target_size=(272, 272))
    if img is None:
        continue
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    pred = net.predict(x)[0]
    top_inds = pred.argsort()[::-1][:5]
    print(f.replace('data\\test\\','').replace('.jpg',''))
    print('    {:.3f}  {}'.format(pred[0], cls_list[0]))
    df=df.append({'ID':f.replace('data\\test\\','').replace('.jpg',''), 'Predicted':'{:.16f}'.format(pred[0])},ignore_index=True)

df

W0818 12:56:27.795692  1900 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0818 12:56:27.796693  1900 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0818 12:56:27.799693  1900 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be rem

['data\\test\\000.jpg', 'data\\test\\001.jpg', 'data\\test\\002.jpg', 'data\\test\\003.jpg', 'data\\test\\004.jpg', 'data\\test\\005.jpg', 'data\\test\\006.jpg', 'data\\test\\007.jpg', 'data\\test\\008.jpg', 'data\\test\\009.jpg', 'data\\test\\010.jpg', 'data\\test\\011.jpg', 'data\\test\\012.jpg', 'data\\test\\013.jpg', 'data\\test\\014.jpg', 'data\\test\\015.jpg', 'data\\test\\016.jpg', 'data\\test\\017.jpg', 'data\\test\\018.jpg', 'data\\test\\019.jpg', 'data\\test\\020.jpg', 'data\\test\\021.jpg', 'data\\test\\022.jpg', 'data\\test\\023.jpg', 'data\\test\\024.jpg', 'data\\test\\025.jpg', 'data\\test\\026.jpg', 'data\\test\\027.jpg', 'data\\test\\028.jpg', 'data\\test\\029.jpg', 'data\\test\\030.jpg', 'data\\test\\031.jpg', 'data\\test\\032.jpg', 'data\\test\\033.jpg', 'data\\test\\034.jpg', 'data\\test\\035.jpg', 'data\\test\\036.jpg', 'data\\test\\037.jpg', 'data\\test\\038.jpg', 'data\\test\\039.jpg', 'data\\test\\040.jpg', 'data\\test\\041.jpg', 'data\\test\\042.jpg', 'data\\tes

000
    1.000  cats
001
    1.000  cats
002
    1.000  cats
003
    0.000  cats
004
    1.000  cats
005
    1.000  cats
006
    0.000  cats
007
    1.000  cats
008
    1.000  cats
009
    1.000  cats
010
    0.000  cats
011
    1.000  cats
012
    0.000  cats
013
    1.000  cats
014
    1.000  cats
015
    0.000  cats
016
    0.000  cats
017
    1.000  cats
018
    0.002  cats
019
    1.000  cats
020
    1.000  cats
021
    1.000  cats
022
    1.000  cats
023
    1.000  cats
024
    1.000  cats
025
    0.000  cats
026
    0.004  cats
027
    0.000  cats
028
    0.000  cats
029
    0.000  cats
030
    0.000  cats
031
    0.000  cats
032
    1.000  cats
033
    1.000  cats
034
    1.000  cats
035
    0.000  cats
036
    1.000  cats
037
    1.000  cats
038
    0.000  cats
039
    1.000  cats
040
    0.000  cats
041
    0.000  cats
042
    1.000  cats
043
    1.000  cats
044
    1.000  cats
045
    0.000  cats
046
    1.000  cats
047
    1.000  cats
048
    0.730  cats
049
    1.000  cats


,ID,Predicted
0,000,1.0000000000000000
1,001,1.0000000000000000
2,002,1.0000000000000000
3,003,0.0000000000000000
4,004,0.9999763965606689
5,005,0.9999997615814209
6,006,0.0000000821091675
7,007,1.0000000000000000
8,008,1.0000000000000000
9,009,1.0000000000000000


In [3]:
df.head()

,ID,Predicted
0,000,1.0000000000000000
1,001,1.0000000000000000
2,002,1.0000000000000000
3,003,0.0000000000000000
4,004,0.9999763965606689


In [4]:
df.to_csv('submission08181200.csv', index=False)